<a href="https://colab.research.google.com/github/pmb2/twilio-ivr-speech-bot/blob/master/OurProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import selenium
import csv
import os
import re
import time
import random
import requests
import captcha_handler
from proxy import get_chromedriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs

# from seleniumwire import webdriver

# http = 'http://proxyusername:proxypassword@{host}:{port}'.format(host=ip, port=port)  # replace proxypassword etc with your proxy user+pass
# https = 'https://proxyusername:proxypassword@{host}:{port}'.format(host=ip, port=port)
# options = {
#     'proxy': {
#         'http': http,
#         'https': https,
#         'no_proxy': 'localhost,127.0.0.1,dev_server:8080'
#     }
# }
# driver = get_chromedriver(use_proxy=True)

driver = webdriver.Chrome(executable_path=r"chromedriver.exe")

with open('Emails.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    first_username = []
    first_password = []
    first_first_name = []
    first_last_name = []

    for line in csv_reader:
        first_username.append(line['Username'])
        first_password.append(line['Password'])
        first_first_name.append(line['FirstName'])
        first_last_name.append(line['LastName'])

username = first_username[0]
password = first_password[0]
first_name = first_first_name[0]
last_name = first_last_name[0]
# print(username)
# print(password)

# with open("Emails.csv", 'r') as f:
#     with open("temp_Emails.csv", 'w', newline='') as f1:
#         writer = csv.writer(f1)
#         writer.writerow(['Username', 'Password'])
#         next(f)
#         next(f)
#         for line in f:
#             f1.write(line)

# os.remove('Emails.csv')
# os.rename('temp_Emails.csv', 'Emails.csv')

driver.get('https://signup.live.com/signup.aspx?mkt=en-gb&lic=1')


WebDriverWait(driver, 30).until(EC.presence_of_element_located(
    (By.ID, 'liveSwitch'))).click()

driver.find_element_by_class_name(
    'ltr_override.liveDomainInput.col-xs-14').send_keys(username)
time.sleep(1)
WebDriverWait(driver, 30).until(
    EC.element_to_be_clickable((By.XPATH, "//*[@id='LiveDomainBoxList']/option[text()='hotmail.com']"))).click()

WebDriverWait(driver, 30).until(
    EC.element_to_be_clickable((By.ID, 'iSignupAction'))).click()

WebDriverWait(driver, 30).until(EC.presence_of_element_located(
    (By.ID, 'PasswordInput'))).send_keys(password)

WebDriverWait(driver, 30).until(
    EC.element_to_be_clickable((By.ID, 'iSignupAction'))).click()

WebDriverWait(driver, 30).until(EC.presence_of_element_located(
    (By.ID, 'FirstName'))).send_keys(first_name)

driver.find_element_by_id('LastName').send_keys(last_name)

WebDriverWait(driver, 30).until(
    EC.element_to_be_clickable((By.ID, 'iSignupAction'))).click()

WebDriverWait(driver, 30).until(EC.presence_of_element_located(
    (By.XPATH, "//*[@id='Country']/option[text()='United States']"))).click()

random_day = random.randint(1, 27)
random_month = random.randint(1, 12)

for i in range(random_day):
    driver.find_element_by_class_name(
        'datepart0.form-control').send_keys(Keys.ARROW_DOWN)

for i in range(random_month):
    driver.find_element_by_class_name(
        'datepart1.form-control').send_keys(Keys.ARROW_DOWN)

driver.find_element_by_class_name(
    'datepart2.form-control').send_keys("1")

for i in range(random_day):
    driver.find_element_by_class_name(
        'datepart2.form-control').send_keys(Keys.ARROW_DOWN)


day = str(random_day)
month = str(random_month)
year = str(1999 - int(random_day))

if len(day) < 2:
    day = "0" + str(random_day)

if len(month) < 2:
    month = "0" + str(random_month)


print('###########DATE TEST#########\n')
print(day, month, year)

WebDriverWait(driver, 30).until(
    EC.element_to_be_clickable((By.ID, 'iSignupAction'))).click()

WebDriverWait(driver, 120).until(EC.presence_of_element_located(
    (By.XPATH, "//iframe[starts-with(@src, 'https://iframe-auth.arkoselabs.com')]")))

driver.switch_to.frame('enforcementFrame')

WebDriverWait(driver, 30).until(EC.presence_of_element_located(
    (By.ID, 'FunCaptcha-Token')))

token = driver.find_element_by_id('FunCaptcha-Token').get_attribute('value')

print("(line 142) token : ", token)

start = token.find('pk=') + 3
end = token.find('|at=', start)
pk = token[start:end]

cap_request = (
    f'https://2captcha.com/in.php?key=a63b7e54cffa09b662f4017ad97262b5&method=funcaptcha&publickey={pk}&surl=https://api.arkoselabs.com&pageurl={driver.current_url}')

r = requests.post(cap_request)
s = requests.Session()
cap_response = (r.text)
cap_id = cap_response.split('OK|', 1)[1]
print("(line 155) cap_id : ", cap_id)
print("Time to get soupy..")
page = requests.get('https://signup.live.com/signup.aspx?mkt=en-gb&lic=1')
soup = bs(page.content, 'html.parser')
content = str(soup)
# x_pattern = re.compile(r'(?<=This is).*?(?=sentence)')
tcxt_pattern = re.compile(r'(?<=,"tcxt":").*?(?="},"WLXAccount":)')
uaid_pattern = re.compile(r'(?<="clientTelemetry":{"uaid":").*?(?=","tcxt")')
canary_pattern = re.compile(r'(?<="apiCanary":").*?(?=","ip":)')
hpgid_pattern = re.compile(r'(?<="hpgid":).*?(?=,"cpgids")')
scid_pattern = re.compile(r'(?<="scid":).*?(?=,"hpgid")')
uiflvr_pattern = re.compile(r'(?<="uiflvr":).*?(?=,"cxhReporting")')
uaid = uaid_pattern.findall(content)[0]
tcxt = tcxt_pattern.findall(content)[0]
canary = canary_pattern.findall(content)[0]
canary = canary.replace(r'\u002f', r'/')
canary = canary.replace(r'\u002b', r'+')
canary = canary.replace(r'\u002a', r':')
hpgid = hpgid_pattern.findall(content)[0]
scid = scid_pattern.findall(content)[0]
uiflvr = uiflvr_pattern.findall(content)[0]

print("\n(line 177) UAID : ", uaid)
print("\n TCXT : ", tcxt)
print("\n CANARY : ", canary)
print("\n HPGID : ", hpgid)
print("\n SCID : ", scid)
print("\n UIFLVR : ", uiflvr)

get_cap_results = (
    f'https://2captcha.com/res.php?key=a63b7e54cffa09b662f4017ad97262b5&action=get&id={cap_id}')

time.sleep(15)

cap_status = (requests.get(get_cap_results).text)

print(cap_status)

for await_cap_results in cap_status:
    if cap_status != ('CAPCHA_NOT_READY'):
        cap_results = cap_status
        break
    time.sleep(5)
    cap_status = (requests.get(get_cap_results).text)
try:
    captcha_token = cap_results.split('OK|', 1)[1]
except NameError:
    print('Error getting captcha results (line 158)')


def funcaptcha_login(self, headers, s, captcha_token, username):
    url = ("https://signup.live.com/API/CreateAccount?mkt=en-gb&lic=1&uaid=" + str(uaid))

    data = {

         "MemberName": str(username) + "@hotmail.com",
         "CheckAvailStateMap": [str(username) + "@hotmail.com:undefined"], "EvictionWarningShown": [],
         "UpgradeFlowToken": {}, "FirstName": first_name, "LastName": last_name, "MemberNameChangeCount": 1,
         "MemberNameAvailableCount": 1, "MemberNameUnavailableCount": 0,
         "CipherValue": "V3KtsdtfUyv8djBq4rN5hRNoCCOFiyt2hC26vEIG8 pXofHyLYEREC5L KqlzRrjFVn5srSkaxwfjjuCV3CGs7csxeh3YGakc6Gb/UEWVB0ueUS GUhipaEKqjzjwKhEDLUNfa2EBFJZcvjf 0PlDMNBjOFojO3S8EviRMSFm1Petyi5vW86mgE45FHHMwoIv4QpMi0DPr D3oWttAmm/ixVNlBidnrOfajPxx9qqwrIOVUZI5VW9HhtAwVt0GjOHUF7lFzd3A 40 PkX9gUDfyiHiU5o/0kOETHFgpcDOfqCnuhn6VdDZk4KPfSJuxa4PfWwmlSkYsJvvEx0SuBBw: =",
         "SKI": "4B8F32B06B3633468A617C4D5781E6B301099447", "BirthDate": f"{day}:{month}:{year}", "Country": "US",
         "IsOptOutEmailDefault": 'true', "IsOptOutEmailShown": 'true', "IsOptOutEmail": 'true', "LW": 'true', "SiteId": "68692",
         "IsRDM": 0, "WReply": 'null', "ReturnUrl": 'null', "SignupReturnUrl": 'null', "uiflvr": uiflvr,
         "uaid": uaid, "SuggestedAccountType": "EASI", "SuggestionType": "Prefer",
         "HFId": "5d315630b108472e8f3d51f0fa0c1d1b", "HType": "enforcement",
         "HSol": captcha_token

        # {"MemberName": "jkcnbaksjbcakjsbcak@hotmail.com",
        #  "CheckAvailStateMap": ["jkcnbaksjbcakjsbcak@hotmail.com:undefined"], "EvictionWarningShown": [],
        #  "UpgradeFlowToken": {}, "FirstName": "khaskhka", "LastName": "poritpoir", "MemberNameChangeCount": 1,
        #  "MemberNameAvailableCount": 1, "MemberNameUnavailableCount": 0,
        #  "CipherValue": "o0cup4Feg8/eRR/07f LDDguV8Y1kqsRYeWuYmtzt1P6hUxwfkQUIoFPToULnMV ILZ7cxhwAEApjWsbyQm4KIghHNvSCpKkT5LXvX8f8c1q3kZX4HgCpHW/cGs0Mvac0ETIi4S /CB7N2Ycq235ZJZp1Vc4IDqn6XVyq1Z7Aq66Gi9uH6JKTmseh0npxtF150qigRkTmcoqQuKZQrhNnWAmqIksVdGoVOSGjXKzccBjlPJ0YhCUY3svQ2KWhuBvYuEq7U0Xx zwXv6rsvW8n01yyIFqwAE9C032S9KVBRLgcXA0hGf89nK2s4nhxHVVfX2TxecXF0/kZgZRziKgog: =",
        #  "SKI": "4B8F32B06B3633468A617C4D5781E6B301099447", "BirthDate": "19:11:1980", "Country": "US",
        #  "IsOptOutEmailDefault": true, "IsOptOutEmailShown": true, "IsOptOutEmail": true, "LW": true, "SiteId": "68692",
        #  "IsRDM": 0, "WReply": null, "ReturnUrl": null, "SignupReturnUrl": null, "uiflvr": 1001,
        #  "uaid": "0a3fbede735242328a8e64b17e5fd999", "SuggestedAccountType": "EASI", "SuggestionType": "Prefer",
        #  "HFId": "fb013b20dbc84980b6435660839421f7", "HType": "enforcement",
        #  "HSol": "9615fed264a5b5bd2.4453957001|r=us-east-1|metabgclr=#ffffff|maintxtclr=#1B1B1B|mainbgclr=#ffffff|guitextcolor=#747474|metaiconclr=#757575|meta=7|lang=en-gb|pk=B7D8911C-5CC8-A9A3-35B0-554ACEE604DA|at=40|ht=1|cdn_url=https://cdn.arkoselabs.com/fc%7Clurl=https://audio-us-east-1.arkoselabs.com%7Csurl=https://api.arkoselabs.com%22,%22HPId%22:%22B7D8911C-5CC8-A9A3-35B0-554ACEE604DA%22,%22scid%22:100118,%22hpgid%22:201040%7D


    }
    try:
        print("Logging in with funcaptcha. (line 173)")
        response = s.post(url, headers=headers, json=data)  # , proxies=proxies
        if response.status_code == 400:
            # error
            return -1
        else:
            return 1
    except:
        return -1


headers = {
    "accept": "application/json",
    "accept-language": "en-US,en;q=0.9",
    "canary": canary,
    "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
    "hpgid": hpgid,
    "scid": scid,
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "tcxt": tcxt,
    "uaid": uaid,
    "uiflvr": uiflvr,
    "x-ms-apitransport": "xhr",
    "x-ms-apiversion": "2",
    "x-requested-with": "XMLHttpRequest"
  }

print("RESPONSE FROM 2CAPTCHA SERVICE: ", captcha_token)
username = username
result = funcaptcha_login(headers=headers, s=s, captcha_token=captcha_token, username=username)  # , proxies
print("Now attempting to get cookies.")
if result == 1:
    print("Successful funcaptcha login")
    for cookie in s.cookies:
        driver.add_cookie({
            'name': cookie.name,
            'value': cookie.value,
            'path': '/',
            'domain': cookie.domain,
        })
    driver.refresh()
    online = True
    time.sleep(3)

else:
    print("Unsuccessful funcaptcha login, Sorry mate!")
    while True:
        time.sleep(1)

# select = driver.find_element_by_id('FunCaptcha-Token')
# driver.execute_script(
#     "arguments[0].value = arguments[1]", select, cap_results2)
#
# select2 = driver.find_element_by_id('verification-token')
# driver.execute_script(
#     "arguments[0].value = arguments[1]", select2, cap_results2)
#
#
# # this is to just check that the values in the two elements got replcaed
# token = driver.find_element_by_id('FunCaptcha-Token').get_attribute('value')
# token2 = driver.find_element_by_id('verification-token').get_attribute('value')
#
# print(token)
# print(token2)


# so at this point both values were replaced, just need to figure out how to submit it with the callback function


# driver.switch_to.default_content
